# 제주 한달살기 숙소 추천을 위한 분석
### 대상 - 펜션, 민박
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- 호텔의 경우 가능 하겠지만 일반적인 한달살기 환경이 아니라는 가정

In [1]:
# !pip install selenium pandas bs4 xlrd requests tqdm openpyxl

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd
import numpy as np

In [7]:
# 야놀자 펜션/풀빌라 - https://www.yanolja.com/sub-home/pension
# 제주 지역 - https://www.yanolja.com/pension/r-900593?advert=AREA&topAdvertiseMore=1

base_url = 'https://www.yanolja.com'
jeju_pension_url = '/pension/r-900593?advert=AREA&topAdvertiseMore=1'

driver = webdriver.Chrome()
driver.get(base_url + jeju_pension_url)
driver.implicitly_wait(10)

#### 실제에서는 스크롤수 확보...
while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(1.0, 1.5), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.3, 1.2), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')

# 펜션리스트 확보

# 초이스랑 나머지로 구분됨
choice = soup.select('.PlaceListItemBanner_container__ARsIm')
other = soup.select('.PlaceListItemText_container__fUIgA')

yanolja_list = [choice, other]

pension_info1 = []
for type_list in yanolja_list:
    for pension in type_list:
        
        pension_name = pension.select_one('.PlaceListTitle_text__2511B').text.strip()

        try :
            pension_score = pension.select_one('.PlaceListScore_rating__3Glxf').text.strip()
            pension_reviewNum = int(pension.select_one('.PlaceListScore_reviewInfo__3QSCU').text.strip()[1:-1])
        except :
            pension_score = np.nan
            pension_reviewNum = np.nan

        try : 
            pension_price = int(pension.select_one('.PlacePriceInfoV2_discountPrice__1PuwK').text.strip().replace(',', ''))
        except :
            pension_price = np.nan

        try: 
            pension_url = pension.attrs['href']
        except:
            pension_url = pension.select_one('a').attrs['href']

        pension_info1.append({'name' : pension_name,
                            'score' : pension_score,
                            'reviewNum' : pension_reviewNum,
                            'price' : pension_price,
                            'link' : pension_url})
        
pension_info1 = pd.DataFrame(pension_info1)
pension_info1.to_csv('./data/data1.csv', index=False)


In [8]:
pension_info1 = pd.read_csv('./data/data1.csv')
pension_info1

,name,score,reviewNum,price,link
0,제주 길리리조트,5.0,67.0,180000.0,/pension/10047368
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",4.3,378.0,129000.0,/pension/3012511
2,제주 애월브릭하우스펜션,5.0,22.0,NaN,/pension/10049007
3,서귀포 블루마운틴4255리조트,4.5,162.0,119000.0,/pension/1000105972
4,제주 애월 씨앤스타펜션(신축펜션),4.7,154.0,NaN,/pension/1000109860
...,...,...,...,...,...
1670,제주 제주소랑앤팜스빌리지펜션,NaN,NaN,NaN,/pension/3004464
1671,제주 다섯손가락펜션,NaN,NaN,90000.0,/pension/3004131
1672,서귀포 해변의집영빈펜션(단독판매),NaN,NaN,100000.0,/pension/3004027
1673,서귀포 행운클로버펜션,NaN,NaN,130000.0,/pension/3003759


In [26]:
pension_info2 = []
driver = webdriver.Chrome()
for sub_url in pension_info1['link']: 
    pension_id = sub_url.split('/')[-1]   
    info_url = f'https://place-site.yanolja.com/places/{pension_id}'
    driver.get(info_url)
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    

    # 숙소 이름
    try:
        pension_name = driver.find_element(By.CSS_SELECTOR, '#__next > div > div > main > article > div.css-1cc3d9 > div.css-6pnu6y > div.css-11vo59c > h1').text.strip()
    except:
        continue
        

    # 주소
    try:
        pension_addr = driver.find_element(By.CLASS_NAME, 'address').text.strip()
    except:
        pension_addr = 'error'

    time.sleep(round(random.uniform(1.0, 2.0), 3))
    # 상세정보 클릭
    driver.find_element(By.CLASS_NAME, 'css-14fkj0f').click()
    driver.execute_script(f'window.scrollTo(0, 0)')
    time.sleep(round(random.uniform(0.0, 1.0), 3))

    # 상세정보 - 소개
    try:
        pension_intro = driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(1) > div > div.css-pw6xk > div').text.strip().replace('\n','')
    except:
        pension_intro = ''
    time.sleep(round(random.uniform(0.5, 1.0), 3))

    # 상세정보 - 편의시설
    facility = driver.find_element(By.CSS_SELECTOR, '#BOTTOM_SHEET > div.css-gqqlqe > div.css-1ulzvpi > div > div:nth-child(2) > div > div.css-pw6xk')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    try:
        pension_text = facility.find_element(By.CLASS_NAME, 'css-7kp13n').text.strip().replace('\n','')
        pension_tag = facility.find_element(By.CLASS_NAME, 'css-bgisrg').text.strip()
    except:
        pension_text = 'error'
        pension_tag = 'error'    

    pension_info2.append({'name' : pension_name,
                          'addr' : pension_addr,
                          'intro' : pension_intro,
                          'tag' : pension_tag,
                          'facility' : pension_text})
    
    time.sleep(round(random.uniform(0.0, 3.0), 3))

pension_info2 = pd.DataFrame(pension_info2)
pension_info2.to_csv('./data/data2.csv', index=False)

    
    

In [27]:
pension_info2 = pd.DataFrame(pension_info2)
pension_info2 

,name,addr,intro,tag,facility
0,제주 길리리조트,제주특별자치도 제주시 한림읍 명재로 100-47,안녕하세요.물빛 예쁜 협재해수욕장 차량 5분 거리의 길리 리조트는 동남아 휴양지인 ...,주차가능\n개별바베큐\n바베큐\n히노끼탕\n와이파이\n독채객실\n객실금연\n어메니티...,개별바비큐● 프라이빗 가든에서 낭만적인 시간과 바비큐를 즐기실 수 있습니다.▷ 빌라...
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",제주특별자치도 제주시 구좌읍 해맞이해안로 2498,,스파/월풀/욕조\n주차가능\n오션뷰\n해수욕장인근\n와이파이\n복층구조\n이벤트가능...,스파/월풀*스파 이용시 주의사항·이용가능 객실 : 코지룸·이용시 소음이 발생하므로 ...
2,제주 애월브릭하우스펜션,"제주특별자치도 제주시 애월읍 신엄로 52 2,3층",'Brick House' 에메랄드빛 애월바다와 한라산이 안아주는 곳.,주차가능\n바베큐\n와이파이\n야외테라스\n객실금연\n커피숍,IPTV/WiFi야외테라스객실금연바비큐장실내 자이글 이용 가능(대여금액 2만원)카페...
3,서귀포 블루마운틴4255리조트,제주특별자치도 서귀포시 성산읍 중산간동로4259,"안녕하세요, 서귀포 블루마운틴풀빌라스파펜션입니다.더없이 맑고 푸른 하늘 아래 그 빛...",스파/월풀/욕조\n주차가능\n수영장\n풀빌라\n바베큐\n불멍\n와이파이\n복층구조\...,스파/월풀· 전객실 자쿠지 (냉수전용)· 이용기간 : 하절기 이용가능· 전 객실 건...
4,제주 애월 씨앤스타펜션(신축펜션),제주특별자치도 제주시 애월읍 애월로3길 17씨앤스타펜션,저녁 노을이 아름다운 애월 한담 카페거리 하이엔드 입구에 위치한 오션뷰 객실의 제주...,주차가능\n오션뷰\n해수욕장인근\n바베큐\n와이파이\n야외테라스,IPTV/WiFi·객실에서 데이터 걱정 없이 빠르고 편리하게 인터넷을 즐겨보세요!·...
...,...,...,...,...,...
1665,제주 제주소랑앤팜스빌리지펜션,제주특별자치도 제주시 한림읍 귀덕7길 17,"이번 제주 여행은, 제주사랑에서 잊지 못할 추억을 만드세요!제주 소랑 앤 팜스 빌리...",주차가능\n바베큐\n독채객실,독채객실바비큐장· 이용 시간 : 18시 ~ 22시· 사용 장소 : 1층 야외바비큐장...
1666,제주 다섯손가락펜션,제주특별자치도 제주시 애월읍 고내로13길 97,편안한 휴식공간 다섯손가락펜션입니다.다섯손가락은 맑은 공기와 편안한 휴식공간이 있는...,주차가능\n바베큐\n독채펜션\n와이파이\n복층구조\n커피숍,독채펜션복층IPTV/WiFi● 무료 wi-fi 이용 가능합니다.● 비밀번호는 TV ...
1667,서귀포 해변의집영빈펜션(단독판매),제주특별자치도 서귀포시 칠십리로 129,눈앞에 펼쳐진 싱그러운 바다가 손에 잡힐 듯.저희 해변의 집 영빈은 앞에는 바다가 ...,주차가능\n해수욕장인근\n와이파이\n야외테라스,IPTV/WiFi야외테라스● 야외 테라스에서 신선한 공기를 느껴보세요~주차가능해수욕...
1668,서귀포 행운클로버펜션,제주특별자치도 서귀포시 표선면 세화강왓로 65,사계절 아름다운 풍경과 편안한 휴식공간이 만들어진 행운클로버펜션에 오신 걸 환영합니...,주차가능\n바베큐\n독채객실,"독채객실바비큐장· 숯불 바비큐 이용 요금은 2~4인 기준 15,000입니다. · 그..."


In [29]:
pension_info2 = pd.read_csv('./data/data2.csv')
pension_info2


,name,addr,intro,tag,facility
0,제주 길리리조트,제주특별자치도 제주시 한림읍 명재로 100-47,안녕하세요.물빛 예쁜 협재해수욕장 차량 5분 거리의 길리 리조트는 동남아 휴양지인 ...,주차가능\n개별바베큐\n바베큐\n히노끼탕\n와이파이\n독채객실\n객실금연\n어메니티...,개별바비큐● 프라이빗 가든에서 낭만적인 시간과 바비큐를 즐기실 수 있습니다.▷ 빌라...
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",제주특별자치도 제주시 구좌읍 해맞이해안로 2498,NaN,스파/월풀/욕조\n주차가능\n오션뷰\n해수욕장인근\n와이파이\n복층구조\n이벤트가능...,스파/월풀*스파 이용시 주의사항·이용가능 객실 : 코지룸·이용시 소음이 발생하므로 ...
2,제주 애월브릭하우스펜션,"제주특별자치도 제주시 애월읍 신엄로 52 2,3층",'Brick House' 에메랄드빛 애월바다와 한라산이 안아주는 곳.,주차가능\n바베큐\n와이파이\n야외테라스\n객실금연\n커피숍,IPTV/WiFi야외테라스객실금연바비큐장실내 자이글 이용 가능(대여금액 2만원)카페...
3,서귀포 블루마운틴4255리조트,제주특별자치도 서귀포시 성산읍 중산간동로4259,"안녕하세요, 서귀포 블루마운틴풀빌라스파펜션입니다.더없이 맑고 푸른 하늘 아래 그 빛...",스파/월풀/욕조\n주차가능\n수영장\n풀빌라\n바베큐\n불멍\n와이파이\n복층구조\...,스파/월풀· 전객실 자쿠지 (냉수전용)· 이용기간 : 하절기 이용가능· 전 객실 건...
4,제주 애월 씨앤스타펜션(신축펜션),제주특별자치도 제주시 애월읍 애월로3길 17씨앤스타펜션,저녁 노을이 아름다운 애월 한담 카페거리 하이엔드 입구에 위치한 오션뷰 객실의 제주...,주차가능\n오션뷰\n해수욕장인근\n바베큐\n와이파이\n야외테라스,IPTV/WiFi·객실에서 데이터 걱정 없이 빠르고 편리하게 인터넷을 즐겨보세요!·...
...,...,...,...,...,...
1665,제주 제주소랑앤팜스빌리지펜션,제주특별자치도 제주시 한림읍 귀덕7길 17,"이번 제주 여행은, 제주사랑에서 잊지 못할 추억을 만드세요!제주 소랑 앤 팜스 빌리...",주차가능\n바베큐\n독채객실,독채객실바비큐장· 이용 시간 : 18시 ~ 22시· 사용 장소 : 1층 야외바비큐장...
1666,제주 다섯손가락펜션,제주특별자치도 제주시 애월읍 고내로13길 97,편안한 휴식공간 다섯손가락펜션입니다.다섯손가락은 맑은 공기와 편안한 휴식공간이 있는...,주차가능\n바베큐\n독채펜션\n와이파이\n복층구조\n커피숍,독채펜션복층IPTV/WiFi● 무료 wi-fi 이용 가능합니다.● 비밀번호는 TV ...
1667,서귀포 해변의집영빈펜션(단독판매),제주특별자치도 서귀포시 칠십리로 129,눈앞에 펼쳐진 싱그러운 바다가 손에 잡힐 듯.저희 해변의 집 영빈은 앞에는 바다가 ...,주차가능\n해수욕장인근\n와이파이\n야외테라스,IPTV/WiFi야외테라스● 야외 테라스에서 신선한 공기를 느껴보세요~주차가능해수욕...
1668,서귀포 행운클로버펜션,제주특별자치도 서귀포시 표선면 세화강왓로 65,사계절 아름다운 풍경과 편안한 휴식공간이 만들어진 행운클로버펜션에 오신 걸 환영합니...,주차가능\n바베큐\n독채객실,"독채객실바비큐장· 숯불 바비큐 이용 요금은 2~4인 기준 15,000입니다. · 그..."


In [30]:
temp = pension_info1[pension_info1['reviewNum'] > 0]
temp

,name,score,reviewNum,price,link
0,제주 길리리조트,5.0,67.0,180000.0,/pension/10047368
1,"제주 루마인펜션(비치썬라이즈,파티-퍼즈바 대여)",4.3,378.0,129000.0,/pension/3012511
2,제주 애월브릭하우스펜션,5.0,22.0,NaN,/pension/10049007
3,서귀포 블루마운틴4255리조트,4.5,162.0,119000.0,/pension/1000105972
4,제주 애월 씨앤스타펜션(신축펜션),4.7,154.0,NaN,/pension/1000109860
...,...,...,...,...,...
1592,서귀포 농장&펜션설문대하우스,5.0,4.0,NaN,/pension/3004642
1602,제주 스타하우스펜션,2.0,35.0,35100.0,/pension/3004740
1612,제주 김녕몽생이펜션,4.5,2.0,188000.0,/pension/3008514
1630,서귀포 데이스테이하우스펜션,3.0,2.0,180000.0,/pension/3008620


In [39]:
temp = pension_info1[pension_info1['reviewNum'] > 0]
temp_stage2 = pd.DataFrame(columns = ['link', 'star', 'text'])

for sub_url in temp['link']:    
    pension_id = sub_url.split('/')[-1]
    review_url = f'https://place-site.yanolja.com/places/{pension_id}/review'
    driver = webdriver.Chrome()
    driver.get(review_url)

    #### 실제에서는 스크롤수 확보...
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(50, 100)})')
        time.sleep(round(random.uniform(1.0, 2.0), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
        time.sleep(round(random.uniform(0.5, 1.0), 3))     
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.5, 1.2), 3))

        # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

    review_soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_list = review_soup.select('.css-nkytaw')

    for review in review_list:
        try: 
            review_text = review.select_one('.css-djrif0').text.strip()
            review_text = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", review_text)

            star_cnt = 0
            star_list = review.select_one('.css-1uvchqn').select('path')
            for star in star_list:
                star_cnt += star.get_attribute_list('fill').count('#fdbd00')
        except:
            review_text = 'error'
            star_cnt = 99


        temp_stage1 = []
        temp_stage1.append({'link' : sub_url,
                            'star' : star_cnt,
                            'text' : review_text})
        temp_stage1 = pd.DataFrame(temp_stage1)

        temp_stage2 = pd.concat([temp_stage2, temp_stage1])
    
    time.sleep(round(random.uniform(1.5, 3.0), 3))

temp_stage2 = pd.DataFrame(temp_stage2)
temp_stage2.to_csv('./data/review.csv', index=False)
temp_stage2

,link,star,text
0,/pension/10047368,5,제가 웬만하면 후기를 잘 안 남기는데 여기는 진짜 꼭 남기고 싶어서 남깁니당 곧 ...
0,/pension/10047368,5,진심으로 다음에 또 오고싶어요 방도 깨끗하고 수건 냄새도 안나고 화장실 바닥도 난방...
0,/pension/10047368,5,객실 들어가자마자 분위기 있는 노래와 빵빵한 에어컨이 틀어져 있어서 놀랐어요 사장님...
0,/pension/10047368,5,재밌게 놀다왔습니다
0,/pension/10047368,5,우선 정말 깨끗하구요 따뜻하고 냄새도 좋고
...,...,...,...
0,/pension/3008514,4,사장님 매우 친절하세요 웃풍이 많이 세서 춥더라구요 하지만 바닥은 따뜻했어요 그리고...
0,/pension/3008620,3,블로그 후기에 친절하시다고 해서 예약을 했어요 어른5인에 애 2인데 4인기준에서 ...
0,/pension/3008620,4,가격대비 너무 좋았구요 사장님이 완전 친절 바로 앞 바다에서 애기들이랑 꽃게도 잡...
0,/pension/3003008,3,가성비 있는 숙소 모기때문에 잠 설치다 불켜서 잠고 잠듬


In [40]:
temp_stage2[temp_stage2['text'] == 'error']

,link,star,text
0,/pension/1000109441,99,error
0,/pension/1000109441,99,error
0,/pension/3011328,99,error
0,/pension/3004718,99,error
0,/pension/3004718,99,error
0,/pension/3004718,99,error
0,/pension/3004718,99,error
0,/pension/3004718,99,error
0,/pension/3004718,99,error
0,/pension/3003099,99,error


In [41]:
temp_stage2

,link,star,text
0,/pension/10047368,5,제가 웬만하면 후기를 잘 안 남기는데 여기는 진짜 꼭 남기고 싶어서 남깁니당 곧 ...
0,/pension/10047368,5,진심으로 다음에 또 오고싶어요 방도 깨끗하고 수건 냄새도 안나고 화장실 바닥도 난방...
0,/pension/10047368,5,객실 들어가자마자 분위기 있는 노래와 빵빵한 에어컨이 틀어져 있어서 놀랐어요 사장님...
0,/pension/10047368,5,재밌게 놀다왔습니다
0,/pension/10047368,5,우선 정말 깨끗하구요 따뜻하고 냄새도 좋고
...,...,...,...
0,/pension/3008514,4,사장님 매우 친절하세요 웃풍이 많이 세서 춥더라구요 하지만 바닥은 따뜻했어요 그리고...
0,/pension/3008620,3,블로그 후기에 친절하시다고 해서 예약을 했어요 어른5인에 애 2인데 4인기준에서 ...
0,/pension/3008620,4,가격대비 너무 좋았구요 사장님이 완전 친절 바로 앞 바다에서 애기들이랑 꽃게도 잡...
0,/pension/3003008,3,가성비 있는 숙소 모기때문에 잠 설치다 불켜서 잠고 잠듬


In [ ]:
### 펜션리스트를 그대로 사용할 것인지?
### 네이버 한달살기 숙소 검색결과를 다시 크롤링 할것인지!
# 이를 통해서 이 숙소들의 위치와 정보를 확인할 것인가....